In [1]:
import numpy as np
y_dict = {}

f = open('data/physionet-a/Outcomes-a.txt', 'r', encoding='utf8')
f.readline()
for line in f:
    str_split = line.split(',')
    stay_day = int(str_split[3])
    y_dict[str_split[0]] = 1 if stay_day < 3 else 0
f.close()

#parameters = {'recordid', 'age', 'gender', 'height', 'icutype', 'weight'}
parameters = {'recordid'}
param_dict = {}

for record_id in y_dict:
    f = open('data/physionet-a/set-a/'+record_id+'.txt', 'r', encoding='utf8')
    f.readline()
    for line in f:
        str_split = line.lower().split(',')
        if str_split[1] not in parameters:
            param_dict[str_split[1]] = len(param_dict)
            parameters.add(str_split[1])
    f.close()

print(len(param_dict), param_dict)


41 {'age': 0, 'gender': 1, 'height': 2, 'icutype': 3, 'weight': 4, 'gcs': 5, 'hr': 6, 'nidiasabp': 7, 'nimap': 8, 'nisysabp': 9, 'resprate': 10, 'temp': 11, 'urine': 12, 'hct': 13, 'bun': 14, 'creatinine': 15, 'glucose': 16, 'hco3': 17, 'mg': 18, 'platelets': 19, 'k': 20, 'na': 21, 'wbc': 22, 'ph': 23, 'paco2': 24, 'pao2': 25, 'diasabp': 26, 'fio2': 27, 'map': 28, 'mechvent': 29, 'sysabp': 30, 'sao2': 31, 'albumin': 32, 'alp': 33, 'alt': 34, 'ast': 35, 'bilirubin': 36, 'lactate': 37, 'cholesterol': 38, 'troponini': 39, 'troponint': 40}


In [2]:
x_dict = {}
for record_id in y_dict:
    x_dict[record_id] = {}
    f = open('data/physionet-a/set-a/'+record_id+'.txt', 'r', encoding='utf8')
    f.readline()
    for line in f:
        str_split = line.lower().split(',')
        time_idx = str_split[0]
        #if time_idx == '00:00':
        #    continue
        if not(time_idx in x_dict[record_id]):
            x_dict[record_id][time_idx] = [0]*len(param_dict)
        if str_split[1] not in param_dict:
            continue
        if float(str_split[2]) > -1:
            x_dict[record_id][time_idx][param_dict[str_split[1]]] = float(str_split[2])
    f.close()

max_t = 0
for record_id in x_dict:
    cur_len = len(x_dict[record_id])
    if cur_len > max_t:
        max_t = cur_len
    if cur_len > 155:
        print(record_id)
print('max time step', max_t)
x_train = []
y_train = []
x_dev = []
y_dev = []
x_test = []
y_test = []
arr = [*x_dict]
np.random.shuffle(arr)
train_arr = set(arr[:3200])
dev_arr = set(arr[3200:3600])
for record_id in x_dict:
    if len(x_dict[record_id]) <= 0: # or len(x_dict[record_id]) >= 155:
        continue
    x_seq = []
    for time_idx in sorted(x_dict[record_id]):
        x_seq += x_dict[record_id][time_idx]
    x_seq = np.array(x_seq).reshape(-1, len(param_dict))

    y_value = y_dict[record_id]
    if record_id in train_arr:
        x_train.append(x_seq)
        y_train.append(np.array([y_value]))
    elif record_id in dev_arr:
        x_test.append(x_seq)
        y_test.append(np.array([y_value]))        
    else:
        x_dev.append(x_seq)
        y_dev.append(np.array([y_value]))         
#print(x_train, y_train)
print(len(x_train), len(x_dev), len(x_test))

133040
133308
133780
133959
134038
135365
135481
136383
137516
139030
139487
139668
140668
140677
141218
max time step 203
3200 400 400


In [3]:
np.save('data/physionet-a/train.x.npy', x_train)
np.save('data/physionet-a/train.y.npy', y_train)
np.save('data/physionet-a/dev.x.npy', x_dev)
np.save('data/physionet-a/dev.y.npy', y_dev)
np.save('data/physionet-a/test.x.npy', x_test)
np.save('data/physionet-a/test.y.npy', y_test)

In [4]:
#d = np.load('data/physionet-a/train.x.npy')